In [ ]:
import My_API_Wraps
import My_FE_Wraps
import My_FS_Wraps
from dask.distributed import Client, LocalCluster
import pandas as pd
import datetime as dt
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import dask.dataframe as dd

In [ ]:
cluster = LocalCluster(
            n_workers=4,
            processes=True,
            threads_per_worker=5,
            memory_limit='6GB',
            dashboard_address=':8787',
            resources={'GPU': 1}
        )
client = Client(cluster)
periods= 1
timeframe= 89
top_coins= 6
api_key= "CG-r57ENE22zzPUiLmjnyFK7YHw"
headers = {
    "accept": "application/json",
    "x-cg-demo-api-key": api_key
    }
client

In [ ]:
df_pandas= My_API_Wraps.CoinGecko_HSPD_Pandas(timeframe, top_coins, periods, api_key)

In [ ]:
EF_pandas_V1= My_FE_Wraps.EF_Pandas_MultiprocessingDistributor(df_pandas, ParameterComplexity=0)

In [ ]:
EF_pandas_V2= My_FE_Wraps.EF_Pandas_DaskDistributor(df_pandas, cluster.scheduler_address, ParameterComplexity=0)


In [ ]:
SF_pandas= My_FS_Wraps.SF_Pandas_Simple(EF_pandas2)

In [ ]:
df_dask = My_API_Wraps.CoinGecko_HSPD_Dask(timeframe=timeframe, top_coins=top_coins, periods=periods, api_key=api_key, splits=5).persist()
EF_dask= My_FE_Wraps.EF_Dask(df_dask, ParameterComplexity=0, splits=5).persist()
SF_dask = My_FS_Wraps.SF_Dask_v1(EF_dask).persist()